In [1]:
#@title Download Pretrained Weights and Install Packages
#for kaggle fix this line
%cd /content/
!rm -rf ./LivePortrait
!git clone https://github.com/KwaiVGI/LivePortrait.git

from tqdm import tqdm
import urllib
import os
import shutil
def conditional_download(url, download_file_path):
    print(f"Downloading {os.path.basename(download_file_path)}")
    base_path = os.path.dirname(download_file_path)

    if not os.path.exists(base_path):
        os.makedirs(base_path)

    if os.path.exists(download_file_path):
        os.remove(download_file_path)

    try:
        request = urllib.request.urlopen(url)  # type: ignore[attr-defined]
        total = int(request.headers.get('Content-Length', 0))
    except urllib.error.URLError as e:
        print(f"Error: Unable to open the URL - {url}")
        print(f"Reason: {e.reason}")
        return

    with tqdm(total=total, desc='Downloading', unit='B', unit_scale=True, unit_divisor=1024) as progress:
        try:
            urllib.request.urlretrieve(url, download_file_path, reporthook=lambda count, block_size, total_size: progress.update(block_size))  # type: ignore[attr-defined]
        except urllib.error.URLError as e:
            print(f"Error: Failed to download the file from the URL - {url}")
            print(f"Reason: {e.reason}")
            return

    print(f"Download successful!")
    print(f"URL: {url}")
    print(f"Save at: {download_file_path}")




def download_models(base_path):
    # Function to download files
    def download_files(file_list, relative_path, base_url):
        for file_name in file_list:
            download_file_path = f"{base_path}/{relative_path}/{file_name}"
            conditional_download(f"{base_url}/{file_name}", download_file_path)

    # Define file lists and paths
    buffalo_l_files = ["2d106det.onnx", "det_10g.onnx"]
    buffalo_l_path = "LivePortrait/pretrained_weights/insightface/models/buffalo_l"
    buffalo_l_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/insightface/models/buffalo_l"

    base_models_files = [
        'appearance_feature_extractor.pth',
        'motion_extractor.pth',
        'spade_generator.pth',
        'warping_module.pth'
    ]
    base_models_path = "LivePortrait/pretrained_weights/liveportrait/base_models"
    base_models_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait/base_models"

    retargeting_models_files = ['stitching_retargeting_module.pth']
    retargeting_models_path = "LivePortrait/pretrained_weights/liveportrait/retargeting_models"
    retargeting_models_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait/retargeting_models"

    landmark_file = ["landmark.onnx"]
    landmark_path = "LivePortrait/pretrained_weights/liveportrait"
    landmark_url = "https://huggingface.co/KwaiVGI/LivePortrait/resolve/main/liveportrait"

    # Download files
    download_files(buffalo_l_files, buffalo_l_path, buffalo_l_url)
    download_files(base_models_files, base_models_path, base_models_url)
    download_files(retargeting_models_files, retargeting_models_path, retargeting_models_url)
    download_files(landmark_file, landmark_path, landmark_url)

#set up base_path
import os
root_path=os.getcwd()
# root_path="/content"
base_path=f"{root_path}"
#download model
download_models(base_path)
os.chdir(f"{base_path}/LivePortrait")
!pip install -r requirements.txt
from IPython.display import clear_output
clear_output()

In [3]:
#@title Gradio app
%cd /content/LivePortrait
import tyro
import gradio as gr
import os.path as osp
from src.utils.helper import load_description
from src.gradio_pipeline import GradioPipeline
from src.config.crop_config import CropConfig
from src.config.argument_config import ArgumentConfig
from src.config.inference_config import InferenceConfig

def partial_fields(target_class, kwargs):
    return target_class(**{k: v for k, v in kwargs.items() if hasattr(target_class, k)})

# Set default values for ArgumentConfig
default_args = {
    "server_name": "0.0.0.0",
    "server_port": 7860,
    "share": True,
    # Add other default values as needed
}

# Create an args object with default values
args = ArgumentConfig(**default_args)

# specify configs for inference
inference_cfg = partial_fields(InferenceConfig, args.__dict__)  # use attribute of args to initial InferenceConfig
crop_cfg = partial_fields(CropConfig, args.__dict__)  # use attribute of args to initial CropConfig
gradio_pipeline = GradioPipeline(
    inference_cfg=inference_cfg,
    crop_cfg=crop_cfg,
    args=args
)
# assets
title_md = "assets/gradio_title.md"
example_portrait_dir = "assets/examples/source"
example_video_dir = "assets/examples/driving"
data_examples = [
    [osp.join(example_portrait_dir, "s9.jpg"), osp.join(example_video_dir, "d0.mp4"), True, True, True, True],
    [osp.join(example_portrait_dir, "s6.jpg"), osp.join(example_video_dir, "d0.mp4"), True, True, True, True],
    [osp.join(example_portrait_dir, "s10.jpg"), osp.join(example_video_dir, "d5.mp4"), True, True, True, True],
    [osp.join(example_portrait_dir, "s5.jpg"), osp.join(example_video_dir, "d6.mp4"), True, True, True, True],
    [osp.join(example_portrait_dir, "s7.jpg"), osp.join(example_video_dir, "d7.mp4"), True, True, True, True],
]
#################### interface logic ####################

# Define components first
eye_retargeting_slider = gr.Slider(minimum=0, maximum=0.8, step=0.01, label="target eyes-open ratio")
lip_retargeting_slider = gr.Slider(minimum=0, maximum=0.8, step=0.01, label="target lip-open ratio")
retargeting_input_image = gr.Image(type="numpy")
output_image = gr.Image(type="numpy")
output_image_paste_back = gr.Image(type="numpy")
output_video = gr.Video()
output_video_concat = gr.Video()

with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.HTML(load_description(title_md))
    gr.Markdown(load_description("assets/gradio_description_upload.md"))
    with gr.Row():
        with gr.Accordion(open=True, label="Source Portrait"):
            image_input = gr.Image(type="filepath")
        with gr.Accordion(open=True, label="Driving Video"):
            video_input = gr.Video()
    gr.Markdown(load_description("assets/gradio_description_animation.md"))
    with gr.Row():
        with gr.Accordion(open=True, label="Animation Options"):
            with gr.Row():
                flag_relative_input = gr.Checkbox(value=True, label="relative motion")
                flag_do_crop_input = gr.Checkbox(value=True, label="do crop")
                flag_remap_input = gr.Checkbox(value=True, label="paste-back")
    with gr.Row():
        with gr.Column():
            process_button_animation = gr.Button("🚀 Animate", variant="primary")
        with gr.Column():
            process_button_reset = gr.ClearButton([image_input, video_input, output_video, output_video_concat], value="🧹 Clear")
    with gr.Row():
        with gr.Column():
            with gr.Accordion(open=True, label="The animated video in the original image space"):
                output_video.render()
        with gr.Column():
            with gr.Accordion(open=True, label="The animated video"):
                output_video_concat.render()
    with gr.Row():
        # Examples
        gr.Markdown("## You could choose the examples below ⬇️")
    with gr.Row():
        gr.Examples(
            examples=data_examples,
            inputs=[
                image_input,
                video_input,
                flag_relative_input,
                flag_do_crop_input,
                flag_remap_input
            ],
            examples_per_page=5
        )
    gr.Markdown(load_description("assets/gradio_description_retargeting.md"))
    with gr.Row():
        eye_retargeting_slider.render()
        lip_retargeting_slider.render()
    with gr.Row():
        process_button_retargeting = gr.Button("🚗 Retargeting", variant="primary")
        process_button_reset_retargeting = gr.ClearButton(
            [
                eye_retargeting_slider,
                lip_retargeting_slider,
                retargeting_input_image,
                output_image,
                output_image_paste_back
            ],
            value="🧹 Clear"
        )
    with gr.Row():
        with gr.Column():
            with gr.Accordion(open=True, label="Retargeting Input"):
                retargeting_input_image.render()
        with gr.Column():
            with gr.Accordion(open=True, label="Retargeting Result"):
                output_image.render()
        with gr.Column():
            with gr.Accordion(open=True, label="Paste-back Result"):
                output_image_paste_back.render()
    # binding functions for buttons
    process_button_retargeting.click(
        fn=gradio_pipeline.execute_image,
        inputs=[eye_retargeting_slider, lip_retargeting_slider],
        outputs=[output_image, output_image_paste_back],
        show_progress=True
    )
    process_button_animation.click(
        fn=gradio_pipeline.execute_video,
        inputs=[
            image_input,
            video_input,
            flag_relative_input,
            flag_do_crop_input,
            flag_remap_input
        ],
        outputs=[output_video, output_video_concat],
        show_progress=True
    )
    image_input.change(
        fn=gradio_pipeline.prepare_retargeting,
        inputs=image_input,
        outputs=[eye_retargeting_slider, lip_retargeting_slider, retargeting_input_image]
    )

##########################################################
demo.launch(share=True)
# demo.launch(
#     server_name=args.server_name,
#     server_port=args.server_port,
#     share=args.share,
# )


/content


[15:22:08] Load appearance_feature_extractor done.                                      ]8;id=247604;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=464277;file:///content/LivePortrait/src/live_portrait_wrapper.py#35\35]8;;\

           Load motion_extractor done.                                                  ]8;id=731075;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=401031;file:///content/LivePortrait/src/live_portrait_wrapper.py#38\38]8;;\

[15:22:09] Load warping_module done.                                                    ]8;id=269527;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=974047;file:///content/LivePortrait/src/live_portrait_wrapper.py#41\41]8;;\

[15:22:10] Load spade_generator done.                                                   ]8;id=381163;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=871146;file:///content/LivePortrait/src/live_portrait_wrapper.py#44\44]8;;\

           Load stitching_retargeting_module done.                                      ]8;id=635168;file:///content/LivePortrait/src/live_portrait_wrapper.py\live_portrait_wrapper.py]8;;\:]8;id=692955;file:///content/LivePortrait/src/live_portrait_wrapper.py#48\48]8;;\

           LandmarkRunner warmup time: 0.179s                                                 ]8;id=529770;file:///content/LivePortrait/src/utils/landmark_runner.py\landmark_runner.py]8;;\:]8;id=884105;file:///content/LivePortrait/src/utils/landmark_runner.py#89\89]8;;\

           FaceAnalysisDIY warmup time: 0.328s                                              ]8;id=202834;file:///content/LivePortrait/src/utils/face_analysis_diy.py\face_analysis_diy.py]8;;\:]8;id=275087;file:///content/LivePortrait/src/utils/face_analysis_diy.py#79\79]8;;\

/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1005: UserWarning: Expected at least 3 arguments for function <bound method GradioPipeline.execute_image of <src.gradio_pipeline.GradioPipeline object at 0x7b9aafc2b760>>, received 2.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1001: UserWarning: Expected 6 arguments for function <bound method GradioPipeline.execute_video of <src.gradio_pipeline.GradioPipeline object at 0x7b9aafc2b760>>, received 5.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/gradio/utils.py:1005: UserWarning: Expected at least 6 arguments for function <bound method GradioPipeline.execute_video of <src.gradio_pipeline.GradioPipeline object at 0x7b9aafc2b760>>, received 5.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://1a4690d7b5f27a9079.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
